In [4]:
import tushare as ts
import pandas as pd

# 1. 设置你的 Tushare Token (请替换成你自己的Token)
TUSHARE_TOKEN = 'a872d82f46046d335ccf68ef591747ff66b9a9d598b40791b80f017a' # <--- 请替换成你的真实TOKEN
pro = ts.pro_api(TUSHARE_TOKEN)

# 2. 定义股票代码和固定的时间段
STOCK_CODE = '000001.SZ'  # 以平安银行为例，你可以换成其他股票代码
START_DATE = '20200101'
END_DATE = '20241231'

print(f"准备获取股票 {STOCK_CODE} 从 {START_DATE} 到 {END_DATE} 的日线数据...")

try:
    # 3. 使用 pro.daily 或 pro_bar 获取日线数据
    df_daily = ts.pro_bar(api=pro,
                          ts_code=STOCK_CODE,
                          start_date=START_DATE,
                          end_date=END_DATE,
                          adj='hfq',       # 后复权 (推荐)
                          asset='E',       # E表示股票
                          freq='D'         # D表示日线
                         )
    # 如果你更喜欢用 pro.daily() 接口，也可以这样:
    # df_daily = pro.daily(ts_code=STOCK_CODE, start_date=START_DATE, end_date=END_DATE)
    # 注意：pro.daily 默认获取的可能是未复权数据或其自身的复权逻辑，
    # pro_bar 在复权类型指定上更直接。使用 hfq (后复权) 通常能保证价格的连续性。

    if df_daily is not None and not df_daily.empty:
        # Tushare 返回的数据可能是日期降序的，我们通常习惯升序
        df_daily = df_daily.sort_values('trade_date', ascending=True).reset_index(drop=True)

        # 4. 打印输出获取到的数据条数和前几条、后几条数据作为验证
        print(f"\n成功获取到股票 {STOCK_CODE} 从 {START_DATE} 到 {END_DATE} 的 {len(df_daily)} 条日线数据。")
        print("\n数据列名:")
        print(df_daily.columns.tolist()) # 打印所有列名
        print("\n数据概览 (info):")
        df_daily.info()
        print("\n前5条数据:")
        print(df_daily.head())
        print("\n后5条数据:")
        print(df_daily.tail())

    else:
        print(f"未能获取到股票 {STOCK_CODE} 在指定时间段的数据，可能原因：")
        print("1. Token无效或积分不足。")
        print("2. 股票在该时段无数据 (例如未上市或已退市)。")
        print("3. 网络问题或API调用限制。")
        if df_daily is not None and df_daily.empty:
            print("API返回了空的数据集。")


except Exception as e:
    print(f"获取数据时发生错误: {e}")

准备获取股票 000001.SZ 从 20200101 到 20241231 的日线数据...

成功获取到股票 000001.SZ 从 20200101 到 20241231 的 1212 条日线数据。

数据列名:
['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close', 'change', 'pct_chg', 'vol', 'amount']

数据概览 (info):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1212 entries, 0 to 1211
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ts_code     1212 non-null   object 
 1   trade_date  1212 non-null   object 
 2   open        1212 non-null   float64
 3   high        1212 non-null   float64
 4   low         1212 non-null   float64
 5   close       1212 non-null   float64
 6   pre_close   1212 non-null   float64
 7   change      1212 non-null   float64
 8   pct_chg     1212 non-null   float64
 9   vol         1212 non-null   float64
 10  amount      1212 non-null   float64
dtypes: float64(9), object(2)
memory usage: 104.3+ KB

前5条数据:
     ts_code trade_date     open     high      low    close  pre_

In [5]:
import psycopg2
from psycopg2 import OperationalError

# 1. 定义数据库连接参数 (请根据你的实际配置修改)
DB_HOST = 'localhost'       # 或者你的 PostgreSQL 服务器 IP 地址
DB_NAME = 'mydb'   # 你创建的数据库名
DB_USER = 'alan-hopiy'          # 你的 PostgreSQL 用户名
DB_PASS = ''      # 你的 PostgreSQL 用户密码
DB_PORT = '5432'            # PostgreSQL 默认端口

conn = None  # 初始化连接变量
cur = None   # 初始化游标变量

try:
    # 2. 尝试建立连接
    print(f"正在连接到 PostgreSQL 数据库 (主机: {DB_HOST}, 数据库: {DB_NAME})...")
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        port=DB_PORT
    )
    print("连接成功！")

    # 3. 创建游标对象
    cur = conn.cursor()

    # 4. 执行简单的 SQL 查询
    print("正在执行查询: SELECT version();")
    cur.execute("SELECT version();")

    # 5. 获取查询结果 (fetchone() 获取一条结果)
    db_version = cur.fetchone()
    if db_version:
        print(f"PostgreSQL 版本: {db_version[0]}")
    else:
        print("未能获取到版本信息。")

except OperationalError as e:
    print(f"数据库连接失败或操作错误: {e}")
    print("请检查以下几点：")
    print("1. PostgreSQL 服务是否已启动？")
    print(f"2. 数据库 '{DB_NAME}' 是否存在？")
    print(f"3. 用户名 '{DB_USER}' 和密码是否正确？")
    print(f"4. 主机名 '{DB_HOST}' 和端口 '{DB_PORT}' 是否正确？")
    print("5. 防火墙设置是否允许连接？")
except Exception as e:
    print(f"发生未知错误: {e}")

finally:
    # 6. 关闭游标和连接 (无论成功与否都应尝试关闭)
    if cur:
        cur.close()
        print("游标已关闭。")
    if conn:
        conn.close()
        print("数据库连接已关闭。")

正在连接到 PostgreSQL 数据库 (主机: localhost, 数据库: mydb)...
连接成功！
正在执行查询: SELECT version();
PostgreSQL 版本: PostgreSQL 17.5 (Postgres.app) on aarch64-apple-darwin23.6.0, compiled by Apple clang version 15.0.0 (clang-1500.3.9.4), 64-bit
游标已关闭。
数据库连接已关闭。


In [6]:
import psycopg2
from psycopg2 import OperationalError, sql # 导入 sql 模块用于安全地构建SQL

# 1. 定义数据库连接参数 (请根据你的实际配置修改)
DB_HOST = 'localhost'       # 或者你的 PostgreSQL 服务器 IP 地址
DB_NAME = 'mydb'   # 你创建的数据库名
DB_USER = 'alan-hopiy'          # 你的 PostgreSQL 用户名
DB_PASS = ''      # 你的 PostgreSQL 用户密码
DB_PORT = '5432'            # PostgreSQL 默认端口

# 定义表名
TABLE_NAME = 'stock_daily_data'

conn = None
cur = None

try:
    # 2. 尝试建立连接
    print(f"正在连接到 PostgreSQL 数据库 (主机: {DB_HOST}, 数据库: {DB_NAME})...")
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        port=DB_PORT
    )
    print("连接成功！")

    # 设置自动提交，这样DDL语句（如CREATE TABLE）会立即生效
    # conn.autocommit = True # 或者在执行后手动 conn.commit()

    cur = conn.cursor()

    # 3. 定义 SQL CREATE TABLE 语句
    # 使用 CREATE TABLE IF NOT EXISTS 避免表已存在时报错
    # 注意：ts_code 和 trade_date 可以考虑作为联合主键，以确保数据的唯一性
    # PRIMARY KEY (ts_code, trade_date)
    # 字段可以根据你从 pro_bar 获取到的实际字段进行调整
    create_table_query = sql.SQL("""
    CREATE TABLE IF NOT EXISTS {table} (
        ts_code VARCHAR(15) NOT NULL,
        trade_date DATE NOT NULL,
        open NUMERIC(10, 2),
        high NUMERIC(10, 2),
        low NUMERIC(10, 2),
        close NUMERIC(10, 2),
        pre_close NUMERIC(10, 2),
        change NUMERIC(10, 2),
        pct_chg NUMERIC(8, 4),
        vol BIGINT,
        amount NUMERIC(18, 3),
        PRIMARY KEY (ts_code, trade_date)
    );
    """).format(table=sql.Identifier(TABLE_NAME)) # 使用 sql.Identifier 来安全地插入表名

    print(f"\n准备创建表 '{TABLE_NAME}'...")
    # print("将要执行的SQL语句:")
    # print(create_table_query.as_string(conn)) # 打印实际执行的SQL (用于调试)

    # 4. 执行 CREATE TABLE 语句
    cur.execute(create_table_query)
    conn.commit() # 提交事务使更改生效

    print(f"表 '{TABLE_NAME}' 创建成功或已存在。")

except OperationalError as e:
    print(f"数据库连接失败或操作错误: {e}")
except Exception as e:
    print(f"创建表时发生错误: {e}")
    if conn:
        conn.rollback() # 如果发生错误，回滚事务
finally:
    # 5. 关闭游标和连接
    if cur:
        cur.close()
    if conn:
        conn.close()
    print("数据库连接已关闭。")

正在连接到 PostgreSQL 数据库 (主机: localhost, 数据库: mydb)...
连接成功！

准备创建表 'stock_daily_data'...
表 'stock_daily_data' 创建成功或已存在。
数据库连接已关闭。


In [11]:
import tushare as ts
import pandas as pd
import psycopg2
from psycopg2 import OperationalError, sql, extras # extras 包含批量插入等辅助功能
import numpy as np # 用于处理 NaN 值

# 1. Tushare Token (请替换成你自己的Token)
TUSHARE_TOKEN = 'a872d82f46046d335ccf68ef591747ff66b9a9d598b40791b80f017a'
pro = ts.pro_api(TUSHARE_TOKEN)

# 2. PostgreSQL 连接参数 (请根据你的实际配置修改)
DB_HOST = 'localhost'       # 或者你的 PostgreSQL 服务器 IP 地址
DB_NAME = 'mydb'   # 你创建的数据库名
DB_USER = 'alan-hopiy'          # 你的 PostgreSQL 用户名
DB_PASS = ''      # 你的 PostgreSQL 用户密码
DB_PORT = '5432'            # PostgreSQL 默认端口

# 3. Tushare 数据获取参数
STOCK_CODE_TO_FETCH = '000001.SZ'
START_DATE_FETCH = '20200101' # 获取少量数据用于测试
END_DATE_FETCH = '20241231'

conn = None
cur = None

def fetch_stock_data_from_tushare(ts_code, start_date, end_date):
    """从Tushare获取股票日线数据"""
    print(f"正在从 Tushare 获取股票 {ts_code} 从 {start_date} 到 {end_date} 的数据...")
    try:
        df = ts.pro_bar(api=pro,
                        ts_code=ts_code,
                        start_date=start_date,
                        end_date=end_date,
                        adj='hfq',
                        asset='E',
                        freq='D',
                        # pro_bar 返回的字段可能比我们表定义的少或多，需要注意匹配
                        # 常见的字段: ts_code, trade_date, open, high, low, close, pre_close, change, pct_chg, vol, amount
                        )
        if df is not None and not df.empty:
            df = df.sort_values('trade_date', ascending=True)
            # 选择并重命名列以匹配数据库表结构 (如果需要)
            # 例如，如果pro_bar返回的vol单位不是手，或amount单位不是千元，需要转换
            # 这里假设pro_bar返回的字段名与我们表设计的字段名大部分一致
            # 处理 NaN/NaT 替换为 None，以便数据库接受为 NULL
            df = df.replace({np.nan: None, pd.NaT: None})
            print(f"成功获取 {len(df)} 条数据。")
            return df
        else:
            print(f"未能从Tushare获取到股票 {ts_code} 的数据。")
            return pd.DataFrame() # 返回空DataFrame
    except Exception as e:
        print(f"从Tushare获取数据时出错: {e}")
        return pd.DataFrame()

try:
    # 获取数据
    stock_df = fetch_stock_data_from_tushare(STOCK_CODE_TO_FETCH, START_DATE_FETCH, END_DATE_FETCH)

    if not stock_df.empty:
        # 连接数据库
        print(f"\n正在连接到 PostgreSQL 数据库 (主机: {DB_HOST}, 数据库: {DB_NAME})...")
        conn = psycopg2.connect(
            host=DB_HOST,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASS,
            port=DB_PORT
        )
        print("数据库连接成功！")
        cur = conn.cursor()

        # 准备插入语句
        # 列的顺序必须与 VALUES 中提供的顺序一致
        # (ts_code, trade_date, open, high, low, close, pre_close, change, pct_chg, vol, amount)
        # ON CONFLICT (ts_code, trade_date) DO NOTHING 表示如果主键冲突（数据已存在），则不执行任何操作
        insert_query = sql.SQL("""
            INSERT INTO {table} (
                ts_code, trade_date, open, high, low, close, pre_close, change, pct_chg, vol, amount
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            ) ON CONFLICT (ts_code, trade_date) DO NOTHING;
        """).format(table=sql.Identifier(TABLE_NAME))

        inserted_rows = 0
        skipped_rows = 0

        print(f"\n准备将数据插入到表 '{TABLE_NAME}'...")
        # 遍历DataFrame的每一行并插入数据
        for index, row in stock_df.iterrows():
            try:
                # 确保row中的字段与INSERT语句中的列一一对应
                # 如果Tushare返回的列名与表列名不完全一致，或缺少某些列，需要在这里适配
                # 例如，如果 pre_close, change, pct_chg 在 pro_bar 的某些情况下可能不存在，要处理好
                data_to_insert = (
                    row.get('ts_code'),
                    row.get('trade_date'), # 确保是 YYYY-MM-DD 格式的日期对象或字符串
                    row.get('open'),
                    row.get('high'),
                    row.get('low'),
                    row.get('close'),
                    row.get('pre_close'), # 如果 pro_bar 没有这个字段，这里可能是 None
                    row.get('change'),    # 同上
                    row.get('pct_chg'),   # 同上
                    row.get('vol'),
                    row.get('amount')
                )
                cur.execute(insert_query, data_to_insert)
                if cur.rowcount > 0: # rowcount > 0 表示插入成功
                    inserted_rows += 1
                else: # rowcount == 0 并且没有异常，通常意味着 ON CONFLICT DO NOTHING 生效了
                    skipped_rows +=1

            except Exception as e_insert:
                print(f"插入行 {index} (日期: {row.get('trade_date')}) 失败: {e_insert}")
                # 可以选择在这里记录失败的行，或者conn.rollback()然后break
                # 为了简单起见，这里我们继续处理下一行

        conn.commit() # 提交所有成功的插入操作
        print(f"\n数据插入完成。成功插入 {inserted_rows} 条新数据。")
        if skipped_rows > 0:
            print(f"由于数据已存在 (ON CONFLICT DO NOTHING)，跳过了 {skipped_rows} 条数据。")

except OperationalError as e_db:
    print(f"数据库操作失败: {e_db}")
    if conn:
        conn.rollback()
except Exception as e_main:
    print(f"发生主流程错误: {e_main}")
    if conn:
        conn.rollback()
finally:
    if cur:
        cur.close()
    if conn:
        conn.close()
    print("数据库连接已关闭。")

正在从 Tushare 获取股票 000001.SZ 从 20200101 到 20241231 的数据...
成功获取 1212 条数据。

正在连接到 PostgreSQL 数据库 (主机: localhost, 数据库: mydb)...
数据库连接成功！

准备将数据插入到表 'stock_daily_data'...

数据插入完成。成功插入 235 条新数据。
由于数据已存在 (ON CONFLICT DO NOTHING)，跳过了 977 条数据。
数据库连接已关闭。


In [12]:
import psycopg2
from psycopg2 import OperationalError, sql
import pandas as pd # 引入pandas，方便将查询结果转为DataFrame展示

# 1. PostgreSQL 连接参数 (请根据你的实际配置修改)
DB_HOST = 'localhost'       # 或者你的 PostgreSQL 服务器 IP 地址
DB_NAME = 'mydb'   # 你创建的数据库名
DB_USER = 'alan-hopiy'          # 你的 PostgreSQL 用户名
DB_PASS = ''      # 你的 PostgreSQL 用户密码
DB_PORT = '5432'            # PostgreSQL 默认端口
TABLE_NAME = 'stock_daily_data'

# 要查询的股票代码
STOCK_CODE_TO_QUERY = '000001.SZ'

conn = None
cur = None

try:
    # 2. 连接数据库
    print(f"正在连接到 PostgreSQL 数据库 (主机: {DB_HOST}, 数据库: {DB_NAME})...")
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        port=DB_PORT
    )
    print("数据库连接成功！")
    cur = conn.cursor()

    # --- 查询一：获取特定股票的总条数 ---
    count_query = sql.SQL("SELECT COUNT(*) FROM {table} WHERE ts_code = %s;").format(
        table=sql.Identifier(TABLE_NAME)
    )
    print(f"\n准备查询股票 {STOCK_CODE_TO_QUERY} 在表 '{TABLE_NAME}' 中的总数据条数...")
    cur.execute(count_query, (STOCK_CODE_TO_QUERY,)) # 参数需要以元组形式传递
    count_result = cur.fetchone() # fetchone() 获取一条结果
    if count_result:
        print(f"股票 {STOCK_CODE_TO_QUERY} 共有 {count_result[0]} 条数据。")
    else:
        print(f"未能查询到股票 {STOCK_CODE_TO_QUERY} 的数据条数。")

    # --- 查询二：获取特定股票的前5条详细数据 ---
    select_query = sql.SQL("SELECT * FROM {table} WHERE ts_code = %s ORDER BY trade_date ASC LIMIT 5;").format(
        table=sql.Identifier(TABLE_NAME)
    )
    # ORDER BY trade_date ASC 确保按日期升序，LIMIT 5 只取前5条
    print(f"\n准备查询股票 {STOCK_CODE_TO_QUERY} 在表 '{TABLE_NAME}' 中的前5条详细数据...")
    cur.execute(select_query, (STOCK_CODE_TO_QUERY,))
    
    # 获取列名
    colnames = [desc[0] for desc in cur.description]
    
    results = cur.fetchall() # fetchall() 获取所有查询结果

    if results:
        print(f"查询到 {len(results)} 条数据：")
        # 将结果转换为Pandas DataFrame以便更好地展示 (可选)
        df_results = pd.DataFrame(results, columns=colnames)
        print(df_results)
    else:
        print(f"未能查询到股票 {STOCK_CODE_TO_QUERY} 的详细数据。")

    # --- 查询三：获取特定股票某个日期之后的数据 (示例) ---
    date_threshold = '20240205'
    select_after_date_query = sql.SQL("""
        SELECT ts_code, trade_date, open, close, pct_chg 
        FROM {table} 
        WHERE ts_code = %s AND trade_date >= %s 
        ORDER BY trade_date ASC;
    """).format(table=sql.Identifier(TABLE_NAME))

    print(f"\n准备查询股票 {STOCK_CODE_TO_QUERY} 在 {date_threshold} 及之后的数据...")
    cur.execute(select_after_date_query, (STOCK_CODE_TO_QUERY, date_threshold))
    
    colnames_partial = [desc[0] for desc in cur.description]
    results_partial = cur.fetchall()

    if results_partial:
        print(f"查询到 {len(results_partial)} 条数据：")
        df_results_partial = pd.DataFrame(results_partial, columns=colnames_partial)
        print(df_results_partial)
    else:
        print(f"未能查询到股票 {STOCK_CODE_TO_QUERY} 在 {date_threshold} 及之后的数据。")


except OperationalError as e_db:
    print(f"数据库操作失败: {e_db}")
except Exception as e_main:
    print(f"发生主流程错误: {e_main}")
finally:
    if cur:
        cur.close()
    if conn:
        conn.close()
    print("数据库连接已关闭。")

正在连接到 PostgreSQL 数据库 (主机: localhost, 数据库: mydb)...
数据库连接成功！

准备查询股票 000001.SZ 在表 'stock_daily_data' 中的总数据条数...
股票 000001.SZ 共有 1212 条数据。

准备查询股票 000001.SZ 在表 'stock_daily_data' 中的前5条详细数据...
查询到 5 条数据：
     ts_code  trade_date     open     high      low    close pre_close  \
0  000001.SZ  2020-01-02  1817.67  1850.42  1806.75  1841.69   1795.84   
1  000001.SZ  2020-01-03  1849.33  1889.72  1847.15  1875.53   1841.69   
2  000001.SZ  2020-01-06  1856.97  1893.00  1846.05  1863.52   1875.53   
3  000001.SZ  2020-01-07  1870.07  1886.45  1850.42  1872.26   1863.52   
4  000001.SZ  2020-01-08  1855.88  1861.34  1815.49  1818.76   1872.26   

   change  pct_chg      vol       amount  
0   45.85   2.5500  1530232  2571196.482  
1   33.84   1.8400  1116195  1914495.474  
2  -12.01  -0.6400   862084  1477930.193  
3    8.74   0.4700   728608  1247047.135  
4  -53.50  -2.8600   847824  1423608.811  

准备查询股票 000001.SZ 在 20240205 及之后的数据...
查询到 218 条数据：
       ts_code  trade_date     open    close

In [1]:
import tushare as ts
import pandas as pd
import psycopg2
from psycopg2 import OperationalError, sql, extras
import numpy as np
import time # 用于延时
from tqdm import tqdm # 用于显示进度条

# --- 配置参数 ---
# 1. Tushare Token (请替换成你自己的Token)
TUSHARE_TOKEN = 'a872d82f46046d335ccf68ef591747ff66b9a9d598b40791b80f017a' # <--- 请替换成你的真实TOKEN
pro = ts.pro_api(TUSHARE_TOKEN)

# 2. PostgreSQL 连接参数
DB_HOST = 'localhost'       # 或者你的 PostgreSQL 服务器 IP 地址
DB_NAME = 'mydb'   # 你创建的数据库名
DB_USER = 'alan-hopiy'          # 你的 PostgreSQL 用户名
DB_PASS = ''      # 你的 PostgreSQL 用户密码
DB_PORT = '5432'            # PostgreSQL 默认端口
TABLE_NAME = 'stock_daily_data'

# 3. 数据获取的时间范围
DATA_START_DATE = '20200101' # 获取数据的统一开始日期
DATA_END_DATE = '20241231'   # 获取数据的统一结束日期

# 4. API 调用延时（秒）- 非常重要！
# 根据你的Tushare账户权限调整。Pro用户限制较宽松。
# 例如，如果 pro_bar 限制为每分钟300次，则每次调用间隔应大于 60/300 = 0.2秒。
# 设置一个保守值，比如 0.25 或 0.3 秒。
API_CALL_DELAY = 0.01 # 秒

# --- 辅助函数 ---
def get_all_a_share_stock_codes():
    """获取所有A股的股票代码列表 (仅ts_code)"""
    print("正在从 Tushare 获取所有 A 股股票代码列表...")
    try:
        # 获取所有正常上市交易的股票列表
        data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name,list_date')
        if data.empty:
            print("未能获取到股票列表。")
            return []
        # 过滤掉上市日期晚于我们数据获取起始日期的股票（可选，但可以减少不必要的API调用）
        # data = data[data['list_date'] <= DATA_START_DATE] # 如果 list_date 是 YYYYMMDD 格式
        print(f"获取到 {len(data)} 只股票。")
        return data['ts_code'].tolist()
    except Exception as e:
        print(f"获取股票列表时发生错误: {e}")
        return []

def fetch_single_stock_data(ts_code, start_date, end_date):
    """从Tushare获取单只股票的日线数据"""
    try:
        df = ts.pro_bar(api=pro,
                        ts_code=ts_code,
                        start_date=start_date,
                        end_date=end_date,
                        adj='hfq',
                        asset='E',
                        freq='D')
        if df is not None and not df.empty:
            df = df.sort_values('trade_date', ascending=True)
            df = df.replace({np.nan: None, pd.NaT: None})
            return df
        return pd.DataFrame()
    except Exception as e:
        # print(f"获取股票 {ts_code} 数据时出错: {e}") # 在主循环中处理日志
        raise e # 将异常向上抛出，由主循环处理

def insert_dataframe_to_db(db_conn, df_to_insert, table_name):
    """将DataFrame数据插入到数据库表中"""
    if df_to_insert.empty:
        return 0, 0

    inserted_count = 0
    skipped_count = 0
    with db_conn.cursor() as cur: # 使用 with 语句确保游标关闭
        # (ts_code, trade_date, open, high, low, close, pre_close, change, pct_chg, vol, amount)
        insert_query = sql.SQL("""
            INSERT INTO {table} (
                ts_code, trade_date, open, high, low, close, pre_close, change, pct_chg, vol, amount
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            ) ON CONFLICT (ts_code, trade_date) DO NOTHING;
        """).format(table=sql.Identifier(table_name))

        for _, row in df_to_insert.iterrows():
            try:
                data_tuple = (
                    row.get('ts_code'), row.get('trade_date'), row.get('open'), row.get('high'),
                    row.get('low'), row.get('close'), row.get('pre_close'), row.get('change'),
                    row.get('pct_chg'), row.get('vol'), row.get('amount')
                )
                cur.execute(insert_query, data_tuple)
                if cur.rowcount > 0:
                    inserted_count += 1
                else:
                    skipped_count += 1
            except Exception as e_insert_row:
                print(f"插入行数据失败 (股票: {row.get('ts_code')}, 日期: {row.get('trade_date')}): {e_insert_row}")
                # 可以考虑将错误信息记录到日志文件
        db_conn.commit()
    return inserted_count, skipped_count

# --- 主流程 ---
if __name__ == '__main__':
    all_ts_codes = get_all_a_share_stock_codes()
    
    # !! 测试阶段：先用少量股票测试 !!
    # all_ts_codes = all_ts_codes[:10] # 例如，只取前10只股票进行测试
    # print(f"\n--- 注意：当前为测试模式，仅处理 {len(all_ts_codes)} 只股票 ---")


    if not all_ts_codes:
        print("没有获取到股票代码，程序退出。")
    else:
        print(f"\n准备处理 {len(all_ts_codes)} 只股票的数据，从 {DATA_START_DATE} 到 {DATA_END_DATE}。")
        print(f"API 调用之间的延时设置为: {API_CALL_DELAY} 秒。")
        
        conn_pg = None
        try:
            conn_pg = psycopg2.connect(host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASS, port=DB_PORT)
            print("数据库连接成功！")

            total_inserted_all_stocks = 0
            total_skipped_all_stocks = 0
            failed_stock_fetches = []

            # 使用 tqdm 创建进度条
            for ts_code in tqdm(all_ts_codes, desc="处理股票进度"):
                # print(f"\n正在处理股票: {ts_code}")
                try:
                    # 1. 从Tushare获取数据
                    # print(f"  正在获取 {ts_code} 从 {DATA_START_DATE} 到 {DATA_END_DATE} 的数据...")
                    stock_df = fetch_single_stock_data(ts_code, DATA_START_DATE, DATA_END_DATE)
                    
                    if not stock_df.empty:
                        # print(f"  获取到 {len(stock_df)} 条数据，准备插入数据库...")
                        inserted, skipped = insert_dataframe_to_db(conn_pg, stock_df, TABLE_NAME)
                        # print(f"  股票 {ts_code}: 成功插入 {inserted} 条, 跳过 {skipped} 条 (已存在)。")
                        total_inserted_all_stocks += inserted
                        total_skipped_all_stocks += skipped
                    else:
                        # print(f"  股票 {ts_code}: 未获取到数据或数据为空。")
                        pass # 可以选择记录这种情况

                except Exception as e_fetch:
                    print(f"  处理股票 {ts_code} 时发生严重错误 (API获取阶段): {e_fetch}")
                    failed_stock_fetches.append(ts_code)
                
                # 2. API 调用延时 - 无论成功与否都应延时
                time.sleep(API_CALL_DELAY)

            print("\n--- 所有股票数据处理完成 ---")
            print(f"总计新插入数据条数: {total_inserted_all_stocks}")
            print(f"总计因已存在而跳过的数据条数: {total_skipped_all_stocks}")
            if failed_stock_fetches:
                print(f"以下股票数据获取失败或处理时发生严重错误 ({len(failed_stock_fetches)} 只):")
                print(failed_stock_fetches)

        except OperationalError as e_db_main:
            print(f"主流程数据库操作失败: {e_db_main}")
        except Exception as e_main_global:
            print(f"主流程发生未知全局错误: {e_main_global}")
        finally:
            if conn_pg:
                conn_pg.close()
                print("数据库连接已关闭。")

正在从 Tushare 获取所有 A 股股票代码列表...


处理股票进度:   0%|          | 0/5413 [00:00<?, ?it/s]

获取到 5413 只股票。

准备处理 5413 只股票的数据，从 20200101 到 20241231。
API 调用之间的延时设置为: 0.01 秒。
数据库连接成功！


处理股票进度:  18%|█▊        | 967/5413 [11:36<52:12,  1.42it/s]  

"None of [Index(['trade_date', 'adj_factor'], dtype='object')] are in the [columns]"


...

插入行数据失败 (股票: 833427.BJ, 日期: 20200204): numeric field overflow
DETAIL:  A field with precision 8, scale 4 must round to an absolute value less than 10^4.

插入行数据失败 (股票: 833427.BJ, 日期: 20200205): current transaction is aborted, commands ignored until end of transaction block

插入行数据失败 (股票: 833427.BJ, 日期: 20200206): current transaction is aborted, commands ignored until end of transaction block

插入行数据失败 (股票: 833427.BJ, 日期: 20200207): current transaction is aborted, commands ignored until end of transaction block

插入行数据失败 (股票: 833427.BJ, 日期: 20200210): current transaction is aborted, commands ignored until end of transaction block

插入行数据失败 (股票: 833427.BJ, 日期: 20200211): current transaction is aborted, commands ignored until end of transaction block

插入行数据失败 (股票: 833427.BJ, 日期: 20200212): current transaction is aborted, commands ignored until end of transaction block

插入行数据失败 (股票: 833427.BJ, 日期: 20200213): current transaction is aborted, commands ignored until end of transaction block

插入行数据失

处理股票进度:  98%|█████████▊| 5315/5413 [1:10:58<00:58,  1.68it/s]

插入行数据失败 (股票: 838227.BJ, 日期: 20220225): numeric field overflow
DETAIL:  A field with precision 8, scale 4 must round to an absolute value less than 10^4.

插入行数据失败 (股票: 838227.BJ, 日期: 20220301): current transaction is aborted, commands ignored until end of transaction block

插入行数据失败 (股票: 838227.BJ, 日期: 20221228): current transaction is aborted, commands ignored until end of transaction block

插入行数据失败 (股票: 838227.BJ, 日期: 20221229): current transaction is aborted, commands ignored until end of transaction block

插入行数据失败 (股票: 838227.BJ, 日期: 20221230): current transaction is aborted, commands ignored until end of transaction block

插入行数据失败 (股票: 838227.BJ, 日期: 20230103): current transaction is aborted, commands ignored until end of transaction block

插入行数据失败 (股票: 838227.BJ, 日期: 20230104): current transaction is aborted, commands ignored until end of transaction block

插入行数据失败 (股票: 838227.BJ, 日期: 20230105): current transaction is aborted, commands ignored until end of transaction block

插入行数据失

处理股票进度: 100%|██████████| 5413/5413 [1:12:00<00:00,  1.27it/s]


--- 所有股票数据处理完成 ---
总计新插入数据条数: 5304776
总计因已存在而跳过的数据条数: 337348
数据库连接已关闭。


In [1]:
import tushare as ts

# --- 配置参数 ---
# 请替换成你自己的Tushare Token
TUSHARE_TOKEN = 'a872d82f46046d335ccf68ef591747ff66b9a9d598b40791b80f017a' # <--- 请务必替换成你的真实TOKEN
pro = ts.pro_api(TUSHARE_TOKEN)

# --- 定义要查询的股票和日期 ---
stocks_to_investigate = [
    {'ts_code': '833427.BJ', 'trade_date': '20200204'},
    {'ts_code': '838227.BJ', 'trade_date': '20220225'},
    # 你可以根据需要添加更多出错的股票和日期
]

print("开始调查特定股票和日期的原始 pct_chg 数据...\n")

for item in stocks_to_investigate:
    ts_code_to_check = item['ts_code']
    date_to_check = item['trade_date']
    
    print(f"正在查询股票: {ts_code_to_check}, 日期: {date_to_check}")
    
    try:
        # 使用 pro_bar 获取指定日期的日线数据
        # 为了获取单日数据，start_date 和 end_date 设置为同一天
        # 我们主要关心 pct_chg 字段，但也获取其他几个字段以便核对
        df_daily = ts.pro_bar(api=pro,
                              ts_code=ts_code_to_check,
                              start_date=date_to_check,
                              end_date=date_to_check,
                              asset='E',       # E表示股票
                              freq='D',        # D表示日线
                              fields='ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount'
                             )
        
        if df_daily is not None and not df_daily.empty:
            print(f"查询成功，获取到 {len(df_daily)} 条记录:")
            # 打印我们关心的字段，特别是 pct_chg
            for index, row in df_daily.iterrows():
                print(f"  股票代码: {row.get('ts_code')}")
                print(f"  交易日期: {row.get('trade_date')}")
                print(f"  开盘价:   {row.get('open')}")
                print(f"  收盘价:   {row.get('close')}")
                print(f"  昨收价:   {row.get('pre_close')}")
                print(f"  涨跌额:   {row.get('change')}")
                print(f"  涨跌幅 (pct_chg): {row.get('pct_chg')}") # <-- 我们重点关注这个值
                print(f"  成交量 (vol):   {row.get('vol')}")
                print(f"  成交额 (amount): {row.get('amount')}")
            print("-" * 30)
        else:
            print(f"未能查询到股票 {ts_code_to_check} 在日期 {date_to_check} 的数据。可能原因：非交易日，或该日无数据。")
            print("-" * 30)
            
    except Exception as e:
        print(f"查询股票 {ts_code_to_check} 数据时发生错误: {e}")
        print("-" * 30)

print("\n调查结束。")

开始调查特定股票和日期的原始 pct_chg 数据...

正在查询股票: 833427.BJ, 日期: 20200204
查询成功，获取到 1 条记录:
  股票代码: 833427.BJ
  交易日期: 20200204
  开盘价:   4.33
  收盘价:   10.96
  昨收价:   nan
  涨跌额:   nan
  涨跌幅 (pct_chg): nan
  成交量 (vol):   80.0
  成交额 (amount): 63.08
------------------------------
正在查询股票: 838227.BJ, 日期: 20220225
查询成功，获取到 1 条记录:
  股票代码: 838227.BJ
  交易日期: 20220225
  开盘价:   20.05
  收盘价:   20.05
  昨收价:   nan
  涨跌额:   nan
  涨跌幅 (pct_chg): nan
  成交量 (vol):   10.0
  成交额 (amount): 20.05
------------------------------

调查结束。


In [2]:
import tushare as ts
import pandas as pd # Tushare 返回 DataFrame，df.iterrows() 会用到

# --- 配置参数 ---
# 请替换成你自己的Tushare Token
TUSHARE_TOKEN = 'a872d82f46046d335ccf68ef591747ff66b9a9d598b40791b80f017a' # <--- 请务必替换成你的真实TOKEN
pro = ts.pro_api(TUSHARE_TOKEN)

# --- 定义要查询的股票和日期 ---
stocks_to_investigate = [
    {'ts_code': '833427.BJ', 'trade_date': '20200204'},
    {'ts_code': '838227.BJ', 'trade_date': '20220225'},
    # 你可以根据需要添加更多出错的股票和日期
]

print("开始调查特定股票和日期的原始数据 (使用后复权 adj='hfq')...\n")

for item in stocks_to_investigate:
    ts_code_to_check = item['ts_code']
    date_to_check = item['trade_date']
    
    print(f"正在查询股票: {ts_code_to_check}, 日期: {date_to_check}")
    
    try:
        # 使用 pro_bar 获取指定日期的日线数据
        # 为了获取单日数据，start_date 和 end_date 设置为同一天
        # 关键：加入了 adj='hfq'
        df_daily = ts.pro_bar(api=pro,
                              ts_code=ts_code_to_check,
                              start_date=date_to_check,
                              end_date=date_to_check,
                              adj='hfq',       # <--- 使用后复权数据
                              asset='E',       # E表示股票
                              freq='D',        # D表示日线
                              fields='ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount'
                             )
        
        if df_daily is not None and not df_daily.empty:
            print(f"查询成功，获取到 {len(df_daily)} 条记录:")
            # 打印我们关心的字段，特别是 pct_chg
            for index, row in df_daily.iterrows(): #  iterrows() 返回 (index, Series) 对
                print(f"  股票代码: {row.get('ts_code')}")
                print(f"  交易日期: {row.get('trade_date')}")
                print(f"  开盘价:   {row.get('open')}")
                print(f"  最高价:   {row.get('high')}") # 添加了 high 和 low
                print(f"  最低价:   {row.get('low')}")
                print(f"  收盘价:   {row.get('close')}")
                print(f"  昨收价:   {row.get('pre_close')}")
                print(f"  涨跌额:   {row.get('change')}")
                print(f"  涨跌幅 (pct_chg): {row.get('pct_chg')}") # <-- 我们重点关注这个值
                print(f"  成交量 (vol):   {row.get('vol')}")
                print(f"  成交额 (amount): {row.get('amount')}")
            print("-" * 40) # 加长了分隔线
        else:
            print(f"未能查询到股票 {ts_code_to_check} 在日期 {date_to_check} 的数据 (adj='hfq')。")
            print("可能原因：非交易日，或该日无数据，或股票在该日期未上市/已退市。")
            print("-" * 40)
            
    except Exception as e:
        print(f"查询股票 {ts_code_to_check} 数据时发生错误: {e}")
        print("-" * 40)

print("\n调查结束。")

开始调查特定股票和日期的原始数据 (使用后复权 adj='hfq')...

正在查询股票: 833427.BJ, 日期: 20200204
查询成功，获取到 1 条记录:
  股票代码: 833427.BJ
  交易日期: 20200204
  开盘价:   433.0
  最高价:   1096.0
  最低价:   433.0
  收盘价:   1096.0
  昨收价:   1.0
  涨跌额:   1095.0
  涨跌幅 (pct_chg): 109500.0
  成交量 (vol):   80.0
  成交额 (amount): 63.08
----------------------------------------
正在查询股票: 838227.BJ, 日期: 20220225
查询成功，获取到 1 条记录:
  股票代码: 838227.BJ
  交易日期: 20220225
  开盘价:   18806.9
  最高价:   18806.9
  最低价:   18806.9
  收盘价:   18806.9
  昨收价:   9.38
  涨跌额:   18797.52
  涨跌幅 (pct_chg): 200400.0
  成交量 (vol):   10.0
  成交额 (amount): 20.05
----------------------------------------

调查结束。


In [3]:
def get_stock_listing_info():
    """获取所有A股的股票代码、上市日期和退市日期"""
    print("正在从 Tushare 获取所有 A 股股票的上市/退市日期信息...")
    try:
        # list_status='L' 获取上市的, 'D' 获取退市的, 'P' 获取暂停上市的
        # 为了全面，可以考虑都获取，或者主要关注 'L' 和 'D'
        # 如果只关心当前还在交易的，就用 list_status='L'，此时 delist_date 通常为 None
        df_l = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name,list_date,delist_date')
        df_d = pro.stock_basic(exchange='', list_status='D', fields='ts_code,name,list_date,delist_date')
        df_p = pro.stock_basic(exchange='', list_status='P', fields='ts_code,name,list_date,delist_date')
        
        data = pd.concat([df_l, df_d, df_p]).drop_duplicates(subset=['ts_code'], keep='first')

        if data.empty:
            print("未能获取到股票列表信息。")
            return []
        
        # 将日期列中的 NaT/None 转换为 None (Python的None) 以便后续处理
        data['delist_date'] = data['delist_date'].astype(object).where(pd.notnull(data['delist_date']), None)

        print(f"获取到 {len(data)} 只股票的上市/退市信息。")
        # 返回一个包含字典的列表，每个字典包含一只股票的信息
        return data[['ts_code', 'list_date', 'delist_date']].to_dict('records')
    except Exception as e:
        print(f"获取股票上市信息时发生错误: {e}")
        return []

In [7]:
import tushare as ts
import pandas as pd
import psycopg2
from psycopg2 import sql, extras
import numpy as np # 用于处理 NaN 值
import time

# --- 配置参数 ---
TUSHARE_TOKEN = 'a872d82f46046d335ccf68ef591747ff66b9a9d598b40791b80f017a' # 替换为你的TOKEN
pro = ts.pro_api(TUSHARE_TOKEN)

DB_HOST = 'localhost'       # 或者你的 PostgreSQL 服务器 IP 地址
DB_NAME = 'mydb'   # 你创建的数据库名
DB_USER = 'alan-hopiy'          # 你的 PostgreSQL 用户名
DB_PASS = ''      # 你的 PostgreSQL 用户密码
DB_PORT = '5432'            # PostgreSQL 默认端口
TABLE_NAME = 'stock_daily_data'

# --- 辅助函数 (假设你已经有了 fetch_single_stock_data 和 insert_dataframe_to_db) ---
# fetch_single_stock_data 应该像这样 (使用 adj='hfq'):
def fetch_single_stock_data(ts_code, start_date, end_date):
    try:
        df = ts.pro_bar(api=pro, ts_code=ts_code, start_date=start_date, end_date=end_date,
                        adj='hfq', asset='E', freq='D')
        if df is not None and not df.empty:
            df = df.sort_values('trade_date', ascending=True)
            # 将Tushare返回的NaN/NaT统一替换为Python的None，便于数据库处理
            df = df.replace({np.nan: None, pd.NaT: None})
            return df
        return pd.DataFrame()
    except Exception as e:
        raise e

# insert_dataframe_to_db (这里不需要截断 pct_chg 的逻辑了，因为非上市数据会被整个移除)
# 但如果上市期间仍有极端值，你可能还是需要之前的截断逻辑，或者修改表结构
def insert_dataframe_to_db(db_conn, df_to_insert, table_name):
    """
    将DataFrame数据插入到数据库表中。
    如果 pct_chg 值超出 NUMERIC(8, 4) 的范围 (±9999.9999)，则进行截断。
    """
    if df_to_insert.empty:
        return 0, 0

    inserted_count = 0
    skipped_count = 0
    
    # NUMERIC(8, 4) 的存储范围边界值
    PCT_CHG_MAX = 9999.9999
    PCT_CHG_MIN = -9999.9999

    with db_conn.cursor() as cur:
        insert_query = sql.SQL("""
            INSERT INTO {table} (
                ts_code, trade_date, open, high, low, close, pre_close, change, pct_chg, vol, amount
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            ) ON CONFLICT (ts_code, trade_date) DO NOTHING;
        """).format(table=sql.Identifier(table_name))

        for _, row in df_to_insert.iterrows():
            original_pct_chg = row.get('pct_chg')
            processed_pct_chg = None # 默认为 None (即数据库中的 NULL)

            if original_pct_chg is not None: # 只有在原始值不是 None 的情况下才处理
                try:
                    val_pct_chg = float(original_pct_chg) # 确保是浮点数
                    
                    if val_pct_chg > PCT_CHG_MAX:
                        processed_pct_chg = PCT_CHG_MAX
                        print(f"  截断警告: 股票 {row.get('ts_code')}, 日期 {row.get('trade_date')}, pct_chg 原值 {val_pct_chg:.4f} 超出上限, 已截断为 {PCT_CHG_MAX}")
                    elif val_pct_chg < PCT_CHG_MIN:
                        processed_pct_chg = PCT_CHG_MIN
                        print(f"  截断警告: 股票 {row.get('ts_code')}, 日期 {row.get('trade_date')}, pct_chg 原值 {val_pct_chg:.4f} 超出下限, 已截断为 {PCT_CHG_MIN}")
                    else:
                        # 保留原始值，但确保是四位小数 (数据库 NUMERIC(8,4) 会自动处理，但这里可以显式round一下，可选)
                        # processed_pct_chg = round(val_pct_chg, 4) 
                        processed_pct_chg = val_pct_chg 
                except (ValueError, TypeError):
                    # 如果 pct_chg 不能转为 float (例如，如果Tushare某天返回了非数字字符串)，则设为 None
                    print(f"  转换警告: 股票 {row.get('ts_code')}, 日期 {row.get('trade_date')}, pct_chg 原值 '{original_pct_chg}' 无法转为数字, 将存为 NULL")
                    processed_pct_chg = None # 保持为 None
            # else: original_pct_chg is already None, so processed_pct_chg remains None

            try:
                data_tuple = (
                    row.get('ts_code'), row.get('trade_date'), row.get('open'), row.get('high'),
                    row.get('low'), row.get('close'), row.get('pre_close'), row.get('change'),
                    processed_pct_chg, # 使用处理（可能被截断）后的 pct_chg 值
                    row.get('vol'), row.get('amount')
                )
                cur.execute(insert_query, data_tuple)
                if cur.rowcount > 0:
                    inserted_count += 1
                else:
                    skipped_count += 1
            except Exception as e_insert_row:
                # 这个打印语句会显示其他可能的数据库插入错误
                print(f"插入行数据失败 (股票: {row.get('ts_code')}, 日期: {row.get('trade_date')}): {e_insert_row}")
                # 如果因为这个错误导致事务中止，后续的 execute 可能都会失败
                # 可以在这里决定是否要 db_conn.rollback() 并为下一行数据开始新事务，但这会使逻辑更复杂
                # 目前的逻辑是，如果发生错误，这个股票的后续行可能也无法插入（如果事务中止）
                # 然后在整个股票处理完后，外层的 try...except 会处理，连接最终会关闭
        
        db_conn.commit() # 在处理完一个DataFrame的所有行后（如果中途没有导致事务中止的错误）提交一次
    return inserted_count, skipped_count

# --- 补写脚本主逻辑 ---
if __name__ == '__main__':
    # 假设我们要补写这两只股票
    stocks_to_process_info = [
        {'ts_code': '833427.BJ', 'list_date': '20201112', 'delist_date': None}, # 示例上市日期, 请用 get_stock_listing_info() 获取真实日期
        {'ts_code': '838227.BJ', 'list_date': '20210603', 'delist_date': None}, # 示例上市日期, 请用 get_stock_listing_info() 获取真实日期
        # 你应该先调用 get_stock_listing_info() 得到所有股票的准确上市/退市日期
        # 然后筛选出你需要补写的股票信息
    ]
    
    # 实际使用时，先获取所有股票的上市信息
    # all_stock_infos = get_stock_listing_info()
    # stocks_to_retry_ts_codes = ['833427.BJ', '838227.BJ'] # 之前失败的股票代码
    # stocks_to_process_info = [info for info in all_stock_infos if info['ts_code'] in stocks_to_retry_ts_codes]


    DATA_START_DATE = '20200101' # 获取数据的整体时间窗口
    DATA_END_DATE = '20241231'

    conn_pg = None
    try:
        conn_pg = psycopg2.connect(host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASS, port=DB_PORT)
        print("数据库连接成功 (用于补写数据)！")

        for stock_info in stocks_to_process_info:
            ts_code = stock_info['ts_code']
            list_date_str = stock_info['list_date'] # 'YYYYMMDD'
            delist_date_str = stock_info['delist_date'] # 'YYYYMMDD' 或 None

            print(f"\n正在为股票 {ts_code} (上市: {list_date_str}, 退市: {delist_date_str}) 补写数据...")
            
            # 1. 获取原始日线数据
            daily_df_raw = fetch_single_stock_data(ts_code, DATA_START_DATE, DATA_END_DATE)

            if not daily_df_raw.empty:
                # 2. 根据上市和退市日期筛选数据
                # 确保 trade_date 是字符串格式以便比较
                daily_df_raw['trade_date'] = daily_df_raw['trade_date'].astype(str)
                
                df_filtered = daily_df_raw[daily_df_raw['trade_date'] >= list_date_str]
                
                if delist_date_str is not None and pd.notna(delist_date_str): # 检查 delist_date 是否有效
                    df_filtered = df_filtered[df_filtered['trade_date'] <= delist_date_str]
                
                original_row_count = len(daily_df_raw)
                filtered_row_count = len(df_filtered)
                print(f"  获取到原始数据 {original_row_count} 条，筛选后（上市期间）剩余 {filtered_row_count} 条。")

                if filtered_row_count > 0:
                    inserted, skipped = insert_dataframe_to_db(conn_pg, df_filtered, TABLE_NAME)
                    print(f"  股票 {ts_code}: 补写成功插入 {inserted} 条, 跳过 {skipped} 条。")
                else:
                    print(f"  股票 {ts_code}: 筛选后无上市期间数据可插入。")
            else:
                print(f"  股票 {ts_code}: 未能获取到数据进行补写。")
            
            time.sleep(0.3) # 补写少量数据时，延时也可以保留，以防万一

    except Exception as e:
        print(f"补写数据过程中发生错误: {e}")
    finally:
        if conn_pg:
            conn_pg.close()
            print("数据库连接已关闭 (补写数据)。")

数据库连接成功 (用于补写数据)！

正在为股票 833427.BJ (上市: 20201112, 退市: None) 补写数据...
  获取到原始数据 1005 条，筛选后（上市期间）剩余 945 条。
  股票 833427.BJ: 补写成功插入 0 条, 跳过 945 条。

正在为股票 838227.BJ (上市: 20210603, 退市: None) 补写数据...
  获取到原始数据 489 条，筛选后（上市期间）剩余 489 条。
  截断警告: 股票 838227.BJ, 日期 20220225, pct_chg 原值 200400.0000 超出上限, 已截断为 9999.9999
  股票 838227.BJ: 补写成功插入 489 条, 跳过 0 条。
数据库连接已关闭 (补写数据)。


In [8]:
import psycopg2
from psycopg2 import OperationalError, sql

# --- PostgreSQL 连接参数 (请根据你的实际配置修改) ---
DB_HOST = 'localhost'       # 或者你的 PostgreSQL 服务器 IP 地址
DB_NAME = 'mydb'   # 你创建的数据库名
DB_USER = 'alan-hopiy'          # 你的 PostgreSQL 用户名
DB_PASS = ''      # 你的 PostgreSQL 用户密码
DB_PORT = '5432'            # PostgreSQL 默认端口
TABLE_NAME = 'stock_daily_data'

TABLE_NAME = 'stock_daily_data' # 你的表名

conn = None
cur = None

print(f"准备查询数据库 '{DB_NAME}' 中表 '{TABLE_NAME}' 的不重复股票数量...")

try:
    # 1. 连接数据库
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        port=DB_PORT
    )
    cur = conn.cursor()
    print("数据库连接成功！")

    # 2. 定义并执行 SQL 查询
    query = sql.SQL("SELECT COUNT(DISTINCT ts_code) FROM {table};").format(
        table=sql.Identifier(TABLE_NAME)
    )
    
    cur.execute(query)
    
    # 3. 获取查询结果
    # count_result 会是一个包含单个元组的列表，例如 [(5000,)]
    # 或者如果用 fetchone()，它会是单个元组 (5000,)
    result = cur.fetchone() 
    
    if result:
        distinct_stock_count = result[0]
        print(f"\n在表 '{TABLE_NAME}' 中，目前共存储了 {distinct_stock_count} 只不同的股票数据。")
    else:
        print("\n未能查询到股票数量（可能是表为空或查询出错）。")

except OperationalError as e_db:
    print(f"数据库操作失败: {e_db}")
except Exception as e_main:
    print(f"发生主流程错误: {e_main}")
finally:
    if cur:
        cur.close()
    if conn:
        conn.close()
    print("数据库连接已关闭。")


准备查询数据库 'mydb' 中表 'stock_daily_data' 的不重复股票数量...
数据库连接成功！

在表 'stock_daily_data' 中，目前共存储了 5370 只不同的股票数据。
数据库连接已关闭。


In [10]:
import tushare as ts
import pandas as pd
import psycopg2
from psycopg2 import OperationalError, sql

# --- 配置参数 ---
TUSHARE_TOKEN = 'a872d82f46046d335ccf68ef591747ff66b9a9d598b40791b80f017a' # <--- 请务必替换成你的真实TOKEN
pro = ts.pro_api(TUSHARE_TOKEN)

DB_HOST = 'localhost'       # 或者你的 PostgreSQL 服务器 IP 地址
DB_NAME = 'mydb'   # 你创建的数据库名
DB_USER = 'alan-hopiy'          # 你的 PostgreSQL 用户名
DB_PASS = ''      # 你的 PostgreSQL 用户密码
DB_PORT = '5432'            # PostgreSQL 默认端口
TABLE_NAME = 'stock_daily_data'
# --- 辅助函数 ---
def get_all_tushare_stock_codes():
    """从Tushare获取所有A股的股票代码列表 (仅ts_code)"""
    print("正在从 Tushare 获取理论上的所有 A 股股票代码列表...")
    try:
        data = pro.stock_basic(exchange='', list_status='L', fields='ts_code') # 只需要 ts_code
        if data.empty:
            print("未能从Tushare获取到股票列表。")
            return set() # 返回空集合
        print(f"从Tushare获取到 {len(data)} 只股票代码。")
        return set(data['ts_code'].tolist()) # 返回集合以便快速求差集
    except Exception as e:
        print(f"从Tushare获取股票列表时发生错误: {e}")
        return set()

def get_distinct_stocks_from_db(db_conn_str, table_name):
    """从数据库获取指定表中所有不重复的ts_code"""
    print(f"正在从数据库表 '{table_name}' 中获取已存储的股票代码列表...")
    conn = None
    cur = None
    db_stock_codes = set()
    try:
        conn = psycopg2.connect(db_conn_str)
        cur = conn.cursor()
        query = sql.SQL("SELECT DISTINCT ts_code FROM {table};").format(
            table=sql.Identifier(table_name)
        )
        cur.execute(query)
        results = cur.fetchall()
        for row in results:
            db_stock_codes.add(row[0])
        print(f"从数据库中获取到 {len(db_stock_codes)} 只不重复的股票代码。")
        return db_stock_codes
    except Exception as e:
        print(f"从数据库获取股票代码时发生错误: {e}")
        return set()
    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()

# --- 主程序入口 ---
if __name__ == '__main__':
    # 构造数据库连接字符串
    db_connection_string = f"host='{DB_HOST}' dbname='{DB_NAME}' user='{DB_USER}' password='{DB_PASS}' port='{DB_PORT}'"

    # 1. 获取Tushare上的全量股票代码
    all_tushare_codes = get_all_tushare_stock_codes()

    if not all_tushare_codes:
        print("无法从Tushare获取股票全量列表，程序终止。")
    else:
        # 2. 获取数据库中已有的股票代码
        db_codes = get_distinct_stocks_from_db(db_connection_string, TABLE_NAME)

        if not db_codes and len(all_tushare_codes) > 0 : # 如果数据库是空的但Tushare列表不是
             print(f"数据库中没有找到任何股票代码，但Tushare列表有 {len(all_tushare_codes)} 个。")
             print("所有Tushare代码都被视为缺失:")
             missing_codes = sorted(list(all_tushare_codes))
        else:
            # 3. 找出差异 (在 all_tushare_codes 中但不在 db_codes 中的)
            missing_codes = sorted(list(all_tushare_codes - db_codes))

        if missing_codes:
            print(f"\n找到了 {len(missing_codes)} 只缺失的股票代码：")
            for code in missing_codes:
                print(code)
            print("\n建议对这些缺失的股票代码进行单独调查，尝试获取其数据。")
        elif len(all_tushare_codes) > 0 and len(db_codes) == len(all_tushare_codes):
             print("\n恭喜！数据库中的股票代码与Tushare列表完全一致，没有缺失的股票。")
        else: # all_tushare_codes 为空的情况已在前面处理
            print("\n数据库中没有数据，也未能从Tushare获取到股票列表进行比较。")

    print("\n查找缺失股票代码的脚本执行完毕。")

正在从 Tushare 获取理论上的所有 A 股股票代码列表...
从Tushare获取到 5413 只股票代码。
正在从数据库表 'stock_daily_data' 中获取已存储的股票代码列表...
从数据库中获取到 5370 只不重复的股票代码。

找到了 43 只缺失的股票代码：
001335.SZ
001356.SZ
001382.SZ
001390.SZ
001395.SZ
001400.SZ
301173.SZ
301275.SZ
301458.SZ
301479.SZ
301501.SZ
301535.SZ
301557.SZ
301560.SZ
301581.SZ
301595.SZ
301601.SZ
301602.SZ
301616.SZ
301629.SZ
301636.SZ
301658.SZ
301662.SZ
301665.SZ
603014.SH
603072.SH
603120.SH
603124.SH
603202.SH
603210.SH
603257.SH
603271.SH
603409.SH
688411.SH
688545.SH
688583.SH
688755.SH
688757.SH
688758.SH
920029.BJ
920068.BJ
920108.BJ
920116.BJ

建议对这些缺失的股票代码进行单独调查，尝试获取其数据。

查找缺失股票代码的脚本执行完毕。


In [11]:
import tushare as ts
import pandas as pd

# --- 配置参数 ---
TUSHARE_TOKEN = 'a872d82f46046d335ccf68ef591747ff66b9a9d598b40791b80f017a' # <--- 请务必替换成你的真实TOKEN
pro = ts.pro_api(TUSHARE_TOKEN)

# --- 定义要调查的股票代码列表 (从你找到的43只中挑选几只) ---
# 示例：我们从你的列表中挑选几只不同类型的
stocks_to_investigate_codes = [
    '001335.SZ', # 深圳主板/中小板的新代码段
    '301173.SZ', # 创业板
    '603014.SH', # 上海主板
    '688411.SH', # 科创板
    '920029.BJ'  # 北交所
]

# 数据获取的时间范围 (与你主脚本一致)
DATA_START_DATE = '20200101'
DATA_END_DATE = '20241231'

print(f"开始调查缺失股票的数据情况 (周期: {DATA_START_DATE} - {DATA_END_DATE})...\n")

for ts_code in stocks_to_investigate_codes:
    print(f"--- 正在调查股票: {ts_code} ---")
    
    # 1. 获取股票的上市日期
    try:
        stock_info = pro.stock_basic(ts_code=ts_code, fields='ts_code,name,list_date')
        if not stock_info.empty:
            list_date = stock_info.iloc[0]['list_date']
            stock_name = stock_info.iloc[0]['name']
            print(f"  股票名称: {stock_name}, 上市日期: {list_date}")
            
            # 简单的日期有效性判断
            if list_date > DATA_END_DATE:
                print(f"  注意: 该股票上市日期 ({list_date}) 晚于我们数据获取的结束日期 ({DATA_END_DATE})，可能无数据。")
            elif list_date > DATA_START_DATE:
                 print(f"  注意: 该股票上市日期 ({list_date}) 在我们数据获取周期内，但不是从周期开始就有数据。")

        else:
            print(f"  未能获取到股票 {ts_code} 的上市日期信息。")
            list_date = None # 设置为None，后续会获取所有数据再看
            
    except Exception as e:
        print(f"  获取股票 {ts_code} 上市日期时发生错误: {e}")
        list_date = None # 获取失败也继续尝试获取日线数据

    # 2. 获取该股票的日线数据 (使用后复权)
    print(f"  正在获取 {ts_code} 的日线数据...")
    try:
        df_daily = ts.pro_bar(api=pro,
                              ts_code=ts_code,
                              start_date=DATA_START_DATE, # 使用完整周期
                              end_date=DATA_END_DATE,
                              adj='hfq',
                              asset='E',
                              freq='D')
        
        if df_daily is not None and not df_daily.empty:
            print(f"  查询成功，获取到 {len(df_daily)} 条日线记录:")
            print("  DataFrame Info:")
            df_daily.info() # 打印列信息，看是否有 trade_date 及其他关键列
            print("\n  DataFrame Head (前5条):")
            print(df_daily.head())
            # 如果需要，也可以打印df_daily.tail()
        elif df_daily is not None and df_daily.empty:
            print(f"  未能查询到股票 {ts_code} 在指定周期内的日线数据 (返回为空表)。")
        else: # df_daily is None
            print(f"  查询股票 {ts_code} 日线数据失败 (API返回None)。")
            
    except Exception as e:
        print(f"  查询股票 {ts_code} 日线数据时发生错误: {e}")
    
    print("-" * 50 + "\n")

print("\n缺失股票调查结束。")

开始调查缺失股票的数据情况 (周期: 20200101 - 20241231)...

--- 正在调查股票: 001335.SZ ---
  股票名称: 信凯科技, 上市日期: 20250415
  注意: 该股票上市日期 (20250415) 晚于我们数据获取的结束日期 (20241231)，可能无数据。
  正在获取 001335.SZ 的日线数据...
  查询股票 001335.SZ 日线数据失败 (API返回None)。
--------------------------------------------------

--- 正在调查股票: 301173.SZ ---
  股票名称: 毓恬冠佳, 上市日期: 20250303
  注意: 该股票上市日期 (20250303) 晚于我们数据获取的结束日期 (20241231)，可能无数据。
  正在获取 301173.SZ 的日线数据...
  查询股票 301173.SZ 日线数据失败 (API返回None)。
--------------------------------------------------

--- 正在调查股票: 603014.SH ---
  股票名称: 威高血净, 上市日期: 20250519
  注意: 该股票上市日期 (20250519) 晚于我们数据获取的结束日期 (20241231)，可能无数据。
  正在获取 603014.SH 的日线数据...
  查询股票 603014.SH 日线数据失败 (API返回None)。
--------------------------------------------------

--- 正在调查股票: 688411.SH ---
  股票名称: 海博思创, 上市日期: 20250127
  注意: 该股票上市日期 (20250127) 晚于我们数据获取的结束日期 (20241231)，可能无数据。
  正在获取 688411.SH 的日线数据...
  查询股票 688411.SH 日线数据失败 (API返回None)。
--------------------------------------------------

--- 正在调查股票: 920029.BJ ---
  股票名称: 开发科技, 上市日期: 20

In [13]:
import tushare as ts
import pandas as pd
import psycopg2
from psycopg2 import OperationalError, sql, extras # extras 用于 IN 子句

# --- 配置参数 ---
TUSHARE_TOKEN = 'a872d82f46046d335ccf68ef591747ff66b9a9d598b40791b80f017a' # <--- 请务必替换成你的真实TOKEN
pro = ts.pro_api(TUSHARE_TOKEN)

DB_HOST = 'localhost'
DB_NAME = 'mydb'
DB_USER = 'alan-hopiy' # 使用你提供的用户名
DB_PASS = ''           # 使用你提供的密码
DB_PORT = '5432'
TABLE_NAME = 'stock_daily_data'

# --- 辅助函数 ---
def get_stock_listing_dates():
    """获取所有A股的股票代码和上市日期"""
    print("正在从 Tushare 获取所有 A 股股票的上市日期信息...")
    try:
        # 我们只需要当前上市的股票，因为我们是要从已有数据中剔除
        data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name,list_date')
        if data.empty:
            print("未能从Tushare获取到股票列表。")
            return pd.DataFrame()
        print(f"从Tushare获取到 {len(data)} 只股票的上市信息。")
        return data[['ts_code', 'list_date', 'name']]
    except Exception as e:
        print(f"从Tushare获取股票列表时发生错误: {e}")
        return pd.DataFrame()

# --- 主程序入口 ---
if __name__ == '__main__':
    all_stocks_df = get_stock_listing_dates()

    if all_stocks_df.empty:
        print("未能获取股票列表，无法继续。")
    else:
        # 筛选出2023年或2024年上市的股票
        # Tushare 的 list_date 格式是 'YYYYMMDD' (字符串)
        all_stocks_df['list_year'] = all_stocks_df['list_date'].str[:4] # 提取年份
        
        stocks_to_remove_df = all_stocks_df[
            (all_stocks_df['list_year'] == '2023') | (all_stocks_df['list_year'] == '2024')
        ]
        
        codes_to_remove = stocks_to_remove_df['ts_code'].tolist()

        if not codes_to_remove:
            print("没有找到在2023年或2024年上市的股票（根据当前Tushare列表）。")
        else:
            print(f"\n找到了 {len(codes_to_remove)} 只在2023年或2024年上市的股票，将从数据库中剔除其数据：")
            # 为了简洁，只打印前20个，你可以取消注释看全部
            # print(codes_to_remove) 
            for i, code in enumerate(codes_to_remove):
                if i < 20:
                    stock_name = stocks_to_remove_df[stocks_to_remove_df['ts_code'] == code]['name'].iloc[0]
                    list_date_val = stocks_to_remove_df[stocks_to_remove_df['ts_code'] == code]['list_date'].iloc[0]
                    print(f"  - {code} ({stock_name}), 上市日期: {list_date_val}")
                elif i == 20:
                    print(f"  ... (以及另外 {len(codes_to_remove) - 20} 只股票)")
                    break
            
            conn = None
            cur = None
            try:
                db_connection_string = f"host='{DB_HOST}' dbname='{DB_NAME}' user='{DB_USER}' password='{DB_PASS}' port='{DB_PORT}'"
                conn = psycopg2.connect(db_connection_string)
                cur = conn.cursor()
                
                # --- 步骤1：先查询这些股票总共有多少条记录将被删除 (安全检查) ---
                if codes_to_remove: # 确保列表不为空
                    # 构建 SQL 查询，例如: SELECT COUNT(*) FROM stock_daily_data WHERE ts_code IN ('code1', 'code2', ...);
                    # 使用 psycopg2.extras.execute_values 或手动构建元组
                    codes_tuple = tuple(codes_to_remove)
                    count_query = sql.SQL("SELECT COUNT(*) FROM {table} WHERE ts_code IN %s;").format(
                        table=sql.Identifier(TABLE_NAME)
                    )
                    cur.execute(count_query, (codes_tuple,))
                    total_rows_to_delete = cur.fetchone()[0]
                    print(f"\n安全检查：这 {len(codes_to_remove)} 只股票在表 '{TABLE_NAME}' 中总共有 {total_rows_to_delete} 条日线数据记录将被删除。")

                    # --- 步骤2：执行删除操作 (默认注释掉，请确认后再取消注释运行) ---
                    confirm_delete = input(f"\n你确定要从数据库表 '{TABLE_NAME}' 中删除这 {total_rows_to_delete} 条记录吗？这是一个不可逆操作！(输入 'yes' 以确认): ")
                    
                    if confirm_delete.lower() == 'yes':
                        print("\n正在执行删除操作...")
                        delete_query = sql.SQL("DELETE FROM {table} WHERE ts_code IN %s;").format(
                            table=sql.Identifier(TABLE_NAME)
                        )
                        cur.execute(delete_query, (codes_tuple,))
                        deleted_row_count = cur.rowcount # DELETE 返回受影响的行数
                        conn.commit() # 提交事务以使删除生效
                        print(f"删除操作完成。成功删除了 {deleted_row_count} 条记录。")
                        
                        # 再次查询确认剩余股票数量
                        remaining_stock_count_query = sql.SQL("SELECT COUNT(DISTINCT ts_code) FROM {table};").format(
                            table=sql.Identifier(TABLE_NAME)
                        )
                        cur.execute(remaining_stock_count_query)
                        remaining_distinct_stocks = cur.fetchone()[0]
                        print(f"操作后，表 '{TABLE_NAME}' 中剩余 {remaining_distinct_stocks} 只不同的股票数据。")
                    else:
                        print("用户取消了删除操作。数据库未做更改。")
                else:
                    print("没有需要删除的股票记录。")

            except OperationalError as e_db:
                print(f"数据库操作失败: {e_db}")
                if conn:
                    conn.rollback() # 如果出错则回滚
            except Exception as e_main:
                print(f"发生主流程错误: {e_main}")
                if conn:
                    conn.rollback()
            finally:
                if cur:
                    cur.close()
                if conn:
                    conn.close()
                print("数据库连接已关闭。")
        
    print("\n脚本执行完毕。")

正在从 Tushare 获取所有 A 股股票的上市日期信息...
从Tushare获取到 5413 只股票的上市信息。

找到了 413 只在2023年或2024年上市的股票，将从数据库中剔除其数据：
  - 001225.SZ (和泰机电), 上市日期: 20230222
  - 001239.SZ (永达股份), 上市日期: 20231212
  - 001260.SZ (坤泰股份), 上市日期: 20230216
  - 001277.SZ (速达股份), 上市日期: 20240903
  - 001278.SZ (一彬科技), 上市日期: 20230308
  - 001279.SZ (强邦新材), 上市日期: 20241011
  - 001282.SZ (三联锻造), 上市日期: 20230524
  - 001286.SZ (陕西能源), 上市日期: 20230410
  - 001287.SZ (中电港), 上市日期: 20230410
  - 001306.SZ (夏厦精密), 上市日期: 20231116
  - 001311.SZ (多利科技), 上市日期: 20230227
  - 001314.SZ (亿道信息), 上市日期: 20230214
  - 001324.SZ (长青科技), 上市日期: 20230522
  - 001326.SZ (联域股份), 上市日期: 20231109
  - 001328.SZ (登康口腔), 上市日期: 20230410
  - 001337.SZ (四川黄金), 上市日期: 20230303
  - 001358.SZ (兴欣新材), 上市日期: 20231221
  - 001359.SZ (平安电工), 上市日期: 20240328
  - 001360.SZ (南矿集团), 上市日期: 20230410
  - 001366.SZ (播恩集团), 上市日期: 20230307
  ... (以及另外 393 只股票)

安全检查：这 413 只股票在表 'stock_daily_data' 中总共有 143304 条日线数据记录将被删除。

你确定要从数据库表 'stock_daily_data' 中删除这 143304 条记录吗？这是一个不可逆操作！(输入 'yes' 以确认): yes


In [39]:
import psycopg2
from psycopg2 import OperationalError, sql
import pandas as pd
import numpy as np
from tqdm import tqdm # 用于显示进度条

# --- 配置参数 ---
DB_HOST = 'localhost'
DB_NAME = 'mydb'
DB_USER = 'alan-hopiy'
DB_PASS = ''
DB_PORT = '5432'
TABLE_NAME = 'stock_daily_data'

BENCHMARK_TS_CODE = '000001.SH' # 上证指数
DATA_START_DATE = '20200101'
DATA_END_DATE = '20241231'
MIN_OBSERVATIONS = 60 # 计算贝塔值所需的最小观测数据点数

conn = None
cur = None
def create_beta_table_if_not_exists(db_conn):
    """如果 stock_beta_values 表不存在，则创建它"""
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS stock_beta_values (
        ts_code VARCHAR(15) NOT NULL,
        benchmark_ts_code VARCHAR(15) NOT NULL,
        start_date DATE NOT NULL,
        end_date DATE NOT NULL,
        beta NUMERIC(10, 4),
        num_observations INTEGER,
        calculation_status VARCHAR(255),
        calculation_run_date TIMESTAMP WITHOUT TIME ZONE DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (ts_code, benchmark_ts_code, start_date, end_date)
    );
    """
    try:
        with db_conn.cursor() as cur:
            cur.execute(create_table_sql)
        db_conn.commit()
        print("表 'stock_beta_values' 已检查/创建完毕。")
    except Exception as e_create_table:
        print(f"创建 'stock_beta_values' 表时发生错误: {e_create_table}")
        db_conn.rollback() # 如果创建失败，回滚
def fetch_daily_close_prices(db_cursor, ts_code, start_date, end_date, table_name):
    """从数据库获取指定代码在时间范围内的每日收盘价"""
    query = sql.SQL("""
        SELECT trade_date, close 
        FROM {table} 
        WHERE ts_code = %s AND trade_date >= %s AND trade_date <= %s
        ORDER BY trade_date ASC;
    """).format(table=sql.Identifier(table_name))
    db_cursor.execute(query, (ts_code, start_date, end_date))
    results = db_cursor.fetchall()
    if not results:
        return pd.DataFrame()
    df = pd.DataFrame(results, columns=['trade_date', 'close'])
    df['trade_date'] = pd.to_datetime(df['trade_date'])
    df = df.set_index('trade_date')
    df['close'] = pd.to_numeric(df['close'])
    return df

def get_all_stock_codes_from_db(db_cursor, table_name, exclude_codes=None):
    """从数据库获取所有不重复的 ts_code，排除指定代码"""
    if exclude_codes is None:
        exclude_codes = []
    
    query_parts = ["SELECT DISTINCT ts_code FROM {table}"]
    params = []
    
    if exclude_codes:
        query_parts.append("WHERE ts_code NOT IN %s")
        params.append(tuple(exclude_codes))
    
    query_parts.append("ORDER BY ts_code ASC;")
    
    final_query_sql = " ".join(query_parts)
    
    query = sql.SQL(final_query_sql).format(table=sql.Identifier(table_name))

    db_cursor.execute(query, params if params else None)
    results = db_cursor.fetchall()
    return [row[0] for row in results]

def insert_betas_to_db(db_conn, df_betas, benchmark_code, period_start, period_end):
    """将计算出的贝塔值插入到数据库的 stock_beta_values 表中"""
    if df_betas.empty:
        print("没有贝塔数据可以插入数据库。")
        return

    # 准备数据元组列表
    # 我们只插入计算状态为 "OK" 且 beta 值非空的记录，或者你可以根据需要调整
    data_tuples = []
    for _, row in df_betas.iterrows():
        if pd.notna(row['beta']) and row['calculation_status'] == 'OK': # 只插入有效的beta
            data_tuples.append((
                row['ts_code'],
                benchmark_code,
                period_start,
                period_end,
                row['beta'],
                row['num_observations'],
                row['calculation_status']
                # calculation_run_date 会使用数据库默认的 CURRENT_TIMESTAMP
            ))
    
    if not data_tuples:
        print("没有有效的贝塔数据可以插入数据库。")
        return

    print(f"准备将 {len(data_tuples)} 条有效贝塔数据插入到 'stock_beta_values' 表...")
    
    # 使用 ON CONFLICT DO UPDATE 来处理已存在的记录 (如果因为某种原因重跑脚本)
    # 更新 beta, num_observations, calculation_status 和 calculation_run_date
    insert_query = """
    INSERT INTO stock_beta_values 
        (ts_code, benchmark_ts_code, start_date, end_date, beta, num_observations, calculation_status) 
    VALUES %s
    ON CONFLICT (ts_code, benchmark_ts_code, start_date, end_date) DO UPDATE SET
        beta = EXCLUDED.beta,
        num_observations = EXCLUDED.num_observations,
        calculation_status = EXCLUDED.calculation_status,
        calculation_run_date = CURRENT_TIMESTAMP;
    """
    try:
        with db_conn.cursor() as cur:
            extras.execute_values(cur, insert_query, data_tuples)
        db_conn.commit()
        print(f"成功将 {len(data_tuples)} 条贝塔数据存入数据库 'stock_beta_values' 表。")
    except Exception as e_insert_db:
        print(f"将贝塔数据存入数据库时发生错误: {e_insert_db}")
        db_conn.rollback()


if __name__ == '__main__':
    all_betas = [] # 用于存储所有股票的贝塔结果
    conn = None # 将 conn 初始化为 None
    try:
        db_connection_string = f"host='{DB_HOST}' dbname='{DB_NAME}' user='{DB_USER}' password='{DB_PASS}' port='{DB_PORT}'"
        conn = psycopg2.connect(db_connection_string)
        cur = conn.cursor()
        print("数据库连接成功！\n")
        create_beta_table_if_not_exists(conn)
        # 1. 获取所有需要计算Beta的股票代码
        print("正在从数据库获取所有股票代码列表...")
        stock_codes_to_calculate = get_all_stock_codes_from_db(cur, TABLE_NAME, exclude_codes=[BENCHMARK_TS_CODE])
        if not stock_codes_to_calculate:
            print("数据库中没有找到需要计算Beta的股票代码。")
        else:
            print(f"将为 {len(stock_codes_to_calculate)} 只股票计算贝塔值。\n")

            # 2. 一次性获取基准指数数据
            print(f"正在获取基准指数 {BENCHMARK_TS_CODE} 的数据...")
            df_benchmark = fetch_daily_close_prices(cur, BENCHMARK_TS_CODE, DATA_START_DATE, DATA_END_DATE, TABLE_NAME)
            if df_benchmark.empty:
                print(f"未能获取到基准指数 {BENCHMARK_TS_CODE} 的数据，无法继续计算贝塔值。")
            else:
                print(f"基准指数数据获取成功，共 {len(df_benchmark)} 条。\n")
                
                # 循环处理每只股票
                for stock_code in tqdm(stock_codes_to_calculate, desc="计算所有股票Beta值"):
                    beta_value = np.nan # 默认为 NaN
                    reason = "OK"

                    df_stock = fetch_daily_close_prices(cur, stock_code, DATA_START_DATE, DATA_END_DATE, TABLE_NAME)

                    if df_stock.empty:
                        reason = "股票数据缺失"
                    else:
                        df_merged = pd.merge(df_stock, df_benchmark, left_index=True, right_index=True, suffixes=('_stock', '_benchmark'))
                        if df_merged.empty or len(df_merged) < 2:
                            reason = "与基准无足够共同交易日"
                        else:
                            df_merged['return_stock'] = df_merged['close_stock'].pct_change()
                            df_merged['return_benchmark'] = df_merged['close_benchmark'].pct_change()
                            
                            df_returns = df_merged[['return_stock', 'return_benchmark']].copy()
                            df_returns = df_returns.replace([np.inf, -np.inf], np.nan)
                            df_returns.dropna(inplace=True)

                            if len(df_returns) < MIN_OBSERVATIONS:
                                reason = f"有效观测点不足({len(df_returns)}<{MIN_OBSERVATIONS})"
                            else:
                                covariance = df_returns['return_stock'].cov(df_returns['return_benchmark'])
                                variance_benchmark = df_returns['return_benchmark'].var()

                                if pd.isna(variance_benchmark) or variance_benchmark == 0 or pd.isna(covariance):
                                    reason = "方差/协方差计算无效"
                                else:
                                    beta_value = covariance / variance_benchmark
                    
                    all_betas.append({'ts_code': stock_code, 'beta': beta_value, 'calculation_status': reason, 'num_observations': len(df_returns) if 'df_returns' in locals() and not df_returns.empty else 0 })
                
                # 将结果转换为 DataFrame
                df_all_betas = pd.DataFrame(all_betas)
                print("\n--- 所有股票贝塔值计算完成 ---")
                
                
                csv_file_path = '/Users/alan-hopiy/Documents/个人研究/基本面量化/all_stock_betas_2020_2024.csv'
                try:
                    df_all_betas.to_csv(csv_file_path, index=False, encoding='utf-8-sig')
                    print(f"\n贝塔值结果已成功保存到 CSV 文件: {csv_file_path}")
                except Exception as e_csv:
                    print(f"\n保存到 CSV 文件时发生错误: {e_csv}")

                # --- 2. 将贝塔值存入数据库 ---
                if conn: # 确保数据库连接是成功的
                    insert_betas_to_db(conn, df_all_betas, BENCHMARK_TS_CODE, DATA_START_DATE, DATA_END_DATE)

    except OperationalError as e_db:
        print(f"数据库操作失败: {e_db}")
    except Exception as e_main:
        print(f"发生主流程错误: {e_main}")
    finally:
        # 关闭游标和连接（确保cur在关闭前已定义）
        # if 'cur' in locals() and cur is not None: # 这种检查方式也可以
        #    cur.close()
        # 由于 cur 是在 conn 成功后才创建，所以只需检查 conn
        if conn:
            conn.close() # psycopg2 的连接对象会自动处理游标关闭
            print("\n数据库连接已关闭。")
        print("脚本执行完毕。")

数据库连接成功！

表 'stock_beta_values' 已检查/创建完毕。
正在从数据库获取所有股票代码列表...


计算所有股票Beta值:   0%|          | 13/4957 [00:00<00:39, 125.91it/s]

将为 4957 只股票计算贝塔值。

正在获取基准指数 000001.SH 的数据...
基准指数数据获取成功，共 1212 条。



计算所有股票Beta值: 100%|██████████| 4957/4957 [00:28<00:00, 172.71it/s]



--- 所有股票贝塔值计算完成 ---

贝塔值结果已成功保存到 CSV 文件: /Users/alan-hopiy/Documents/个人研究/基本面量化/all_stock_betas_2020_2024.csv
准备将 4957 条有效贝塔数据插入到 'stock_beta_values' 表...
成功将 4957 条贝塔数据存入数据库 'stock_beta_values' 表。

数据库连接已关闭。
脚本执行完毕。


In [38]:
import psycopg2
from psycopg2 import sql

# --- 数据库连接配置 ---
DB_HOST = 'localhost'
DB_NAME = 'mydb'
DB_USER = 'alan-hopiy'
DB_PASS = ''  # 空字符串密码
DB_PORT = '5432'

# --- 删除条件 ---
# 北交所股票通常以 .BJ 结尾
STOCK_SUFFIX_BJ = '%.BJ'
# 目标日期，删除此日期之前的数据
# 假设 trade_date 在数据库中是以 'YYYYMMDD' 格式的字符串存储
TARGET_DATE_STR = '20211115'

def delete_bj_early_data():
    """
    连接到 PostgreSQL 数据库并删除北交所股票
    在指定日期之前的历史数据。
    """
    conn = None
    cursor = None
    deleted_rows_count = 0

    try:
        # 1. 建立数据库连接
        conn = psycopg2.connect(
            host=DB_HOST,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASS,
            port=DB_PORT
        )
        conn.autocommit = False # 关闭自动提交，手动管理事务
        cursor = conn.cursor()

        # 2. 构造 SQL 查询语句
        # 使用 psycopg2.sql 来安全地构造查询
        count_query = sql.SQL("""
            SELECT COUNT(*)
            FROM stock_daily_data
            WHERE ts_code LIKE {suffix}
            AND trade_date < {date}
        """).format(
            suffix=sql.Literal(STOCK_SUFFIX_BJ),
            date=sql.Literal(TARGET_DATE_STR)
        )
        
        delete_query = sql.SQL("""
            DELETE FROM stock_daily_data
            WHERE ts_code LIKE {suffix}
            AND trade_date < {date}
        """).format(
            suffix=sql.Literal(STOCK_SUFFIX_BJ),
            date=sql.Literal(TARGET_DATE_STR)
        )
        
        # (重要步骤) 在执行删除前，先查询有多少条记录会被删除
        print(f"正在查询 'stock_daily_data' 表中 ts_code LIKE '{STOCK_SUFFIX_BJ}' 且 trade_date < '{TARGET_DATE_STR}' 的记录数量...")
        cursor.execute(count_query)
        count_to_delete = cursor.fetchone()[0]
        print(f"查询结果：将有 {count_to_delete} 条北交所股票在 {TARGET_DATE_STR} 之前的数据计划被删除。")

        if count_to_delete > 0:
            # 请求用户最终确认
            confirm = input(f"您确认要从数据库 '{DB_NAME}' 的 'stock_daily_data' 表中永久删除这 {count_to_delete} 条记录吗？(yes/no): ")
            if confirm.lower() == 'yes':
                # 3. 执行删除操作
                print("正在执行删除操作...")
                cursor.execute(delete_query)
                deleted_rows_count = cursor.rowcount # 获取受影响的行数

                # 4. 提交事务
                conn.commit()
                print(f"成功：已从 stock_daily_data 表中删除了 {deleted_rows_count} 条记录。")
            else:
                print("操作已取消，未改动任何数据。")
                conn.rollback() # 回滚事务
        else:
            print("没有符合条件的记录需要删除。")
            conn.rollback() # 即使没有数据删除，也确保事务结束


    except psycopg2.Error as e:
        if conn:
            conn.rollback() # 如果发生错误，回滚事务
        print(f"数据库操作失败: {e}")
        # print(f"PGCode: {e.pgcode}, PGError: {e.pgerror}") # 更详细的错误信息
    except Exception as e:
        if conn:
            conn.rollback()
        print(f"发生了一个预料之外的错误: {e}")
    finally:
        # 5. 关闭游标和连接
        if cursor:
            cursor.close()
        if conn:
            conn.close()
        print("数据库连接已关闭。")

if __name__ == '__main__':
    print("脚本将尝试连接数据库并执行数据清理操作。")
    print(f"目标数据库: '{DB_NAME}' on '{DB_HOST}:{DB_PORT}', 用户: '{DB_USER}'")
    print(f"操作表: 'stock_daily_data'")
    print(f"清理条件: 'ts_code' LIKE '{STOCK_SUFFIX_BJ}' AND 'trade_date' < '{TARGET_DATE_STR}' (假设trade_date为YYYYMMDD格式字符串)")
    print("-" * 50)
    print("重要提示：此脚本会先查询将要删除的记录数量，并请求最终确认。")
    print("请确保您已了解操作后果，强烈建议提前备份数据。")
    print("-" * 50)
    
    delete_bj_early_data()

脚本将尝试连接数据库并执行数据清理操作。
目标数据库: 'mydb' on 'localhost:5432', 用户: 'alan-hopiy'
操作表: 'stock_daily_data'
清理条件: 'ts_code' LIKE '%.BJ' AND 'trade_date' < '20211115' (假设trade_date为YYYYMMDD格式字符串)
--------------------------------------------------
重要提示：此脚本会先查询将要删除的记录数量，并请求最终确认。
请确保您已了解操作后果，强烈建议提前备份数据。
--------------------------------------------------
正在查询 'stock_daily_data' 表中 ts_code LIKE '%.BJ' 且 trade_date < '20211115' 的记录数量...
查询结果：将有 29027 条北交所股票在 20211115 之前的数据计划被删除。
您确认要从数据库 'mydb' 的 'stock_daily_data' 表中永久删除这 29027 条记录吗？(yes/no): yes
正在执行删除操作...
成功：已从 stock_daily_data 表中删除了 29027 条记录。
数据库连接已关闭。
